In [199]:
# Import des bibliothèques de code
import re
import os
import time

import folium
import pandas as pd

from folium.plugins import HeatMap
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from PIL import Image

In [204]:
def screenshot_map(m, filename="heatmap"):
    # Sauvegarder temporairement en HTML
    m.save(f"{filename}.html")
    
    # Set up headless browser
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--window-size=1200,800")
    driver = webdriver.Chrome(options=options)
    
    # Charger la carte
    driver.get("file://" + os.path.abspath(f"{filename}_temp.html"))
    time.sleep(3)  # Attendre qu'elle charge
    
    # Prendre une capture d'écran Infractions entrainant la mort
    driver.save_screenshot(f"{filename}_full_temp.png")
    driver.quit()

    # Couper environ 50 px des côtés pour retirer le UI de la capture
    img = Image.open(f"{filename}_full.png")
    width, height = img.size
    cropped = img.crop((50, 0, width - 50, height))
    cropped.save(f"figures/{filename}.png")
    
    # Supprimer fichiers temporaires
    os.remove(f"{filename}_temp.html")
    os.remove(f"{filename}_full_temp.png")

In [192]:
def create_heatmap_from_data(df, map_center, lang_filter= list()):
    filtered_df = df[df['Origine_linguistique'].isin(lang_filter)] if lang_filter else df
    m = folium.Map(location=map_center, zoom_start=6)
    heat_data = filtered_df[['Latitude', 'Longitude']].values.tolist()
    HeatMap(heat_data, radius=15, blur=10, max_zoom=1, min_opacity=0.3).add_to(m)
    return m

In [193]:
# Import des données
df = pd.read_csv('donnees/250505_DQ_DonneesOuvertes_ToponymesDesofficialises_TableDonnees_v1.csv')  

# Calcul du centre de la carte (moyenne de toutes les coordonnées)
map_center = [df['Latitude'].mean(), df['Longitude'].mean()]

# Déclaration des filtres de langue
langues_autochtones = [
    'Langue amérindienne indéterminée', 'Abénaquis', 'Algonquin', 'Attikamek',
    'Cri', 'Malécite', 'Micmac', 'Innu', 'Mohawk', 'Naskapis', 'Wendat', 'Inuktitut'
]
langues_coloniales = ["Français", "Anglais"]

In [194]:
# Créer les cartes pour chaque analyse qui nous intéresse
m = create_heatmap_from_data(df, map_center)
m2 = create_heatmap_from_data(df, map_center, lang_filter=langues_autochtones)
m3 = create_heatmap_from_data(df, map_center, lang_filter=langues_coloniales)

In [202]:
#Créer les captures d'écran des trois cartes
screenshot_map(m, filename="250509_WB_DonneesOuvertes_Toponymes_HeatmapComplete_v1")
screenshot_map(m2, filename="250509_WB_DonneesOuvertes_Toponymes_HeatmapLanguesAutochtones_v1")
screenshot_map(m3, filename="250509_WB_DonneesOuvertes_Toponymes_HeatmapFrancaisAnglais_v1")